Train an imitator using the best hyperparameters from multiple optuna studies. The Hyperparameters were chosen using time series splits, while this agent will be trained on the entire dataset.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset

import numpy as np

import pandas as pd
import os

In [2]:
BATCH = 2**3
INPUTS = 31
DEVICE = 'cuda'
EPOCHS = 1000
SEED = 42
DATADIR = r'Karla\3-26-5 PPOc Karla results' +'/'
DATASET = 'baseline_obs-a.csv'
SAVEDIR = DATADIR + 'proxies' + '/'
NAME = 'imitator'

In [3]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)
        self.targets = torch.tensor(df.iloc[:, -1].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx].unsqueeze(dim=-1)

In [4]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)
        self.targets = torch.tensor(df.iloc[:, -1].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx].unsqueeze(dim=-1)
    
def load_SA_dataset():
    #path to data directory
    data_dir = os.path.join(os.getcwd(), DATADIR) #run this from citylearn folder
    data_dir = os.path.normpath(data_dir) #resolve '..'
    #load features
    df_data = pd.read_csv(data_dir + '/baseline_obs-a.csv',
                    header=0,
                    index_col=0,
                    dtype='float32')
    df_data.set_index(df_data.index.astype(int), inplace=True)
    return df_data

In [5]:
# def get_data_subset(dataset,train_ids, valid_ids, batch_size):
#     """returns data with no shuffling as timeseries samples are NOT iid
#     validation data is provided as a single batch"""
#     train_subsampler = Subset(dataset, 
#                               train_ids)
#     valid_subsampler = Subset(dataset, 
#                               valid_ids)
#     train_loader = torch.utils.data.DataLoader(train_subsampler,
#                                               batch_size=batch_size,)
#     valid_loader = torch.utils.data.DataLoader(valid_subsampler, 
#                                              batch_size=len(valid_subsampler),
#                                              )
#     #assert len(valid_loader) == 1, 'validation data is not a single batch'
#     return train_loader, valid_loader

Load training data

In [6]:
df_SA = pd.read_csv(DATADIR+DATASET,
                    header=0,
                    index_col=0)
sa_dataset = MyDataset(df=df_SA)
data_loader = torch.utils.data.DataLoader(sa_dataset,
                                          batch_size=BATCH,)

Def model

In [7]:
model = nn.Sequential(
    nn.Linear(INPUTS, 464),
    nn.ReLU(),
    nn.Dropout(0.25),
    nn.Linear(464,192),
    nn.ReLU(),
    nn.Dropout(0.45),
    nn.Linear(192, 1)
).to(DEVICE)

In [8]:
optimizer = optim.SGD(model.parameters(),
                          lr=0.0622)

Set random seed for reproducability

In [9]:
torch.manual_seed = SEED

Train and evaluation on entire dataset

In [10]:
best_rmse = np.inf
best_epoch = None
best_params = None
loss_fn = nn.HuberLoss()

for epoch in range(EPOCHS):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):

        data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

        optimizer.zero_grad()
        output = model(data) #output is 2d, with each pred a single item list, targets is 1d
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

    # Validation of the model using RMSE, so the result is always comparable
    model.eval()
    #correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader): #should run only once
            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            pred = model(data)
            rmse = torch.sqrt(F.mse_loss(pred,target))
    
    #best model...        
    if rmse < best_rmse:
        best_rmse = rmse
        best_epoch = epoch
        best_params = model.state_dict().copy()

    print(f'Epoch: {epoch}, RMSE: {rmse}')

print(f'Best epoch: {best_epoch}, best RMSE {best_rmse}')

Epoch: 0, RMSE: 0.1253700703382492
Epoch: 1, RMSE: 0.11872057616710663
Epoch: 2, RMSE: 0.11740599572658539
Epoch: 3, RMSE: 0.11771854758262634
Epoch: 4, RMSE: 0.11926814168691635
Epoch: 5, RMSE: 0.11973816901445389
Epoch: 6, RMSE: 0.1209409087896347
Epoch: 7, RMSE: 0.12361063808202744
Epoch: 8, RMSE: 0.12254507094621658
Epoch: 9, RMSE: 0.12476077675819397
Epoch: 10, RMSE: 0.12644070386886597
Epoch: 11, RMSE: 0.12773706018924713
Epoch: 12, RMSE: 0.1268978714942932
Epoch: 13, RMSE: 0.12818002700805664
Epoch: 14, RMSE: 0.12685757875442505
Epoch: 15, RMSE: 0.12918131053447723
Epoch: 16, RMSE: 0.12888556718826294
Epoch: 17, RMSE: 0.1290365308523178
Epoch: 18, RMSE: 0.12938447296619415
Epoch: 19, RMSE: 0.12870512902736664
Epoch: 20, RMSE: 0.12811487913131714
Epoch: 21, RMSE: 0.1311454027891159
Epoch: 22, RMSE: 0.13017933070659637
Epoch: 23, RMSE: 0.13079528510570526
Epoch: 24, RMSE: 0.13256143033504486
Epoch: 25, RMSE: 0.13216660916805267
Epoch: 26, RMSE: 0.12886402010917664
Epoch: 27, RMSE:

In [12]:
torch.save(model, SAVEDIR + NAME + '.pth')

In [13]:
model.load_state_dict(best_params)
torch.save(model, SAVEDIR + NAME + ' best.pth')